In [ ]:
!pip install moviepy

в аудио первые 60 сек

In [6]:
from moviepy.editor import VideoFileClip
import os

def video_to_audio_1min(video_path, audio_path, audio_format='mp3'):
    """
    Converts the first 1 minute of a video file to an audio file using MoviePy.

    Parameters:
    - video_path: Path to the input video file.
    - audio_path: Path to save the output audio file.
    - audio_format: The desired audio format ('mp3', 'wav', etc.). Default is 'mp3'.

    Example:
    video_to_audio_1min('input_video.mp4', 'output_audio.mp3', 'mp3')
    """
    try:
        # Check if the input video file exists
        if not os.path.exists(video_path):
            print(f"Input video file {video_path} does not exist.")
            return

        # Load the video file
        video_clip = VideoFileClip(video_path)

        # Set the duration to the first 60 seconds (1 minute)
        if video_clip.duration > 60:
            video_clip = video_clip.subclip(0, 60)

        # Extract the audio
        audio_clip = video_clip.audio

        # Write the audio file
        audio_clip.write_audiofile(
            audio_path,
            codec='libmp3lame' if audio_format == 'mp3' else None,
            logger=None  # Suppress verbose output
        )

        # Close the clips
        audio_clip.close()
        video_clip.close()

        print(f"Audio extracted and saved as {audio_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
video_file = 'IMG_5367.MP4'
audio_file = 'output_audio.mp3'  # Change the extension based on desired format
audio_format = 'mp3'  # 'mp3', 'wav', etc.
video_to_audio_1min(video_file, audio_file, audio_format)


Audio extracted and saved as output_audio.mp3


расшифровочка

In [ ]:
import pandas as pd
import subprocess
from faster_whisper import WhisperModel
import os

# Load your DataFrame (replace with your actual DataFrame)
# df = pd.read_csv('your_dataframe.csv')

# Example DataFrame for demonstration purposes
# df = pd.DataFrame({
#     'video_path': ['path/to/video1.mp4', 'path/to/video2.mp4'],
#     'target': [0, 1]
# })

# Load the Faster Whisper model (you can choose 'tiny', 'base', 'small', 'medium', 'large-v2')
# Specify device as 'cpu' or 'cuda' depending on your setup
model_size = "base"  # Change this as needed
device = "cpu"       # Change to 'cuda' if you have a GPU

model = WhisperModel(model_size, device=device, compute_type="float16")

def process_video(video_path):
    try:
        # Temporary filenames for audio and cropped video
        audio_file = "temp_audio.wav"
        cropped_video_file = "temp_video.mp4"
        
        # Get the duration of the video using ffprobe
        cmd_duration = [
            "ffprobe", "-v", "error", "-show_entries",
            "format=duration", "-of",
            "default=noprint_wrappers=1:nokey=1", video_path
        ]
        duration = float(subprocess.check_output(cmd_duration).decode().strip())
        
        # If the video is longer than 1 minute, crop it to 60 seconds
        if duration > 60:
            # Crop the video to 60 seconds
            cmd_crop = [
                "ffmpeg", "-y", "-i", video_path, "-t", "60",
                "-c", "copy", cropped_video_file
            ]
            subprocess.run(cmd_crop, check=True)
            input_video = cropped_video_file
        else:
            input_video = video_path
        
        # Extract audio from the video
        cmd_audio = [
            "ffmpeg", "-y", "-i", input_video, "-vn",
            "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1",
            audio_file
        ]
        subprocess.run(cmd_audio, check=True)
        
        # Transcribe the audio using Faster Whisper
        segments, info = model.transcribe(audio_file, beam_size=5)
        text = "".join([segment.text for segment in segments])
        
        # Clean up temporary files
        if os.path.exists(audio_file):
            os.remove(audio_file)
        if os.path.exists(cropped_video_file) and input_video == cropped_video_file:
            os.remove(cropped_video_file)
        
        return text.strip()
    except Exception as e:
        print(f"Error processing {video_path}: {e}")
        return ""
    
# Apply the function to each video in the DataFrame
df['subs'] = df['video_path'].apply(process_video)


средний фрейм

In [ ]:
from moviepy.editor import VideoFileClip
from PIL import Image
import numpy as np

def extract_middle_frame(video_path, output_image_path):
    """
    Извлекает серединный кадр из видео и сохраняет его как изображение.

    Параметры:
    - video_path: Путь к входному видеофайлу.
    - output_image_path: Путь для сохранения выходного изображения.

    Пример:
    extract_middle_frame('input_video.mp4', 'middle_frame.jpg')
    """
    try:
        # Загружаем видеофайл
        video_clip = VideoFileClip(video_path)

        # Вычисляем время середины видео
        middle_time = video_clip.duration / 2

        # Получаем кадр в середине видео
        frame = video_clip.get_frame(middle_time)

        # Сохраняем кадр как изображение
        image = Image.fromarray(frame)
        image.save(output_image_path)

        # Закрываем видео
        video_clip.close()

        print(f"Серединный кадр извлечен и сохранен как {output_image_path}")

    except Exception as e:
        print(f"Произошла ошибка: {e}")

# Пример использования:
video_file = 'input_video.mp4'
output_image = 'middle_frame.jpg'
extract_middle_frame(video_file, output_image)


клип + берт

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load X-CLIP model and tokenizer
video_model_name = "microsoft/xclip-base-patch32"
video_tokenizer = AutoTokenizer.from_pretrained(video_model_name)
video_model = AutoModel.from_pretrained(video_model_name)


In [ ]:
from sentence_transformers import SentenceTransformer

# Load LaBSE Russian model
text_model_name = "cointegrated/LaBSE-en-ru"
text_model = SentenceTransformer(text_model_name)


In [ ]:
import pandas as pd

# Example DataFrame
# df should have columns: 'video_path' and 'text'
df = pd.DataFrame({
    'video_path': ['path/to/video1.mp4', 'path/to/video2.mp4', 'path/to/video3.mp4'],
    'text': ['Первый текст', 'Второй текст', 'Третий текст']
})


In [ ]:
import torchvision.transforms as transforms
import torchvision.io as io
from tqdm import tqdm

def extract_video_embedding(video_path, model, device):
    # Read video frames
    video, _, info = io.read_video(video_path, pts_unit='sec')
    video = video.permute(0, 3, 1, 2)  # Rearrange dimensions to (frames, channels, height, width)

    # Resize and normalize frames
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                             std=[0.26862954, 0.26130258, 0.27577711])
    ])

    video = preprocess(video.float() / 255.0)  # Normalize pixel values

    # Move to device
    video = video.to(device)

    # Get embeddings
    with torch.no_grad():
        outputs = model(inputs_embeds=video.unsqueeze(0))
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

    return embedding


In [ ]:
def extract_text_embedding(text, model):
    embedding = model.encode(text)
    return embedding

In [ ]:
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
video_model.to(device)

video_embeddings = []
text_embeddings = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    video_path = row['video_path']
    text = row['text']

    # Extract video embedding
    video_embedding = extract_video_embedding(video_path, video_model, device)
    video_embeddings.append(video_embedding)

    # Extract text embedding
    text_embedding = extract_text_embedding(text, text_model)
    text_embeddings.append(text_embedding)

# Convert lists to arrays
video_embeddings = np.array(video_embeddings)
text_embeddings = np.array(text_embeddings)


In [ ]:
# Ensure that both embeddings have the same length along the first dimension
assert video_embeddings.shape[0] == text_embeddings.shape[0], "Mismatch in number of samples."

# Concatenate embeddings
combined_embeddings = np.concatenate((video_embeddings, text_embeddings), axis=1)

# Prepare labels (assuming you have labels)
labels = df['label'].values  # Replace 'label' with your actual label column


In [ ]:
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# Convert data to PyTorch tensors
X = torch.tensor(combined_embeddings, dtype=torch.float32)
y = torch.tensor(labels, dtype=torch.long)

# Create a dataset and dataloader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )
    def forward(self, x):
        return self.layers(x)

# Set parameters
input_dim = combined_embeddings.shape[1]
hidden_dim = 256  # You can adjust this
num_classes = len(np.unique(labels))  # Number of classes in your dataset

model = MLP(input_dim, hidden_dim, num_classes)
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
# Training loop
num_epochs = 10  # Adjust the number of epochs

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_X.size(0)

    epoch_loss = running_loss / len(dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    

предикт

In [ ]:
def predict(video_path, text, video_model, text_model, mlp_model, device):
    # Extract video embedding
    video_embedding = extract_video_embedding(video_path, video_model, device)
    
    # Extract text embedding
    text_embedding = extract_text_embedding(text, text_model)
    
    # Concatenate embeddings
    combined_embedding = np.concatenate((video_embedding, text_embedding))
    
    # Convert to PyTorch tensor and move to device
    input_tensor = torch.tensor(combined_embedding, dtype=torch.float32).unsqueeze(0).to(device)
    
    # Set model to evaluation mode
    mlp_model.eval()
    
    with torch.no_grad():
        # Get model output
        output = mlp_model(input_tensor)
        
        # Apply softmax to get probabilities
        probabilities = torch.softmax(output, dim=1).cpu().numpy()[0]
        
        # Get predicted class
        predicted_class = np.argmax(probabilities)
    
    return predicted_class, probabilities


In [ ]:
# New video and text inputs
new_video_path = 'path/to/new_video.mp4'
new_text = 'Новый текст для предсказания'

# Perform prediction
predicted_class, probabilities = predict(new_video_path, new_text, video_model, text_model, model, device)

print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")
